In [ ]:
from scipy.interpolate import RectBivariateSpline
from dms2dec.dms_convert import dms2dec
from shapely.geometry import Polygon
from rasterio.io import MemoryFile
from shapely.ops import transform
from rasterio.plot import show
from rasterio.mask import mask
from skimage import exposure
from rasterio import plot
from glob import glob
import numpy as np
import rasterio
import pyproj
import os

import matplotlib.pyplot as plt

In [ ]:
def reproject(polygons, proj_from, proj_to):
    proj_from = pyproj.Proj(proj_from)
    proj_to = pyproj.Proj(proj_to)
    
    projection = pyproj.Transformer.from_proj(proj_from, proj_to)
    return [transform(projection.transform, p) for p in polygons]

In [ ]:
def crop_memory_tiff_file(mem_file, polygons, crop):
    polygons = reproject(polygons, "EPSG:4326", mem_file.crs)  # "EPSG:4326"
    result_image, result_transform = mask(mem_file, polygons, crop=crop)
    
    profile = mem_file.profile
    profile.update(width=result_image.shape[1], 
                   height=result_image.shape[2], 
                   transform=result_transform)

    result_f = MemoryFile().open(**profile)
    result_f.write(result_image)
    
    return result_f

In [ ]:
def extrapolate(arr, target_dim):
    x = np.array(range(arr.shape[0]))
    y = np.array(range(arr.shape[1]))
    z = arr
    xx = np.linspace(x.min(), x.max(), target_dim[0])
    yy = np.linspace(y.min(), y.max(), target_dim[1])

    new_kernel = RectBivariateSpline(x, y, z, kx=2, ky=2)
    result = new_kernel(xx, yy)

    return result

In [ ]:
def get_test_polygon(swap_coordinates=False):
    my_points = [
        (dms2dec('''44°37'16"N'''), dms2dec('''8°34'54"E''')),
        (dms2dec('''44°37'16"N'''), dms2dec('''8°36'26"E''')),
        (dms2dec('''44°35'49"N'''), dms2dec('''8°36'26"E''')),
        (dms2dec('''44°35'49"N'''), dms2dec('''8°34'54"E'''))
    ]
    
    # Copernicus hub likes polygons in lng/lat format
    return Polygon([(y, x) if swap_coordinates else (x, y) for x, y in my_points])

In [ ]:
src_root_data_dir = # Sentinel2 data folder
tiff_root_data_dir = # output data folder

bands = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]              # definire le bande che si vogliono utilizzare
resolutions = ["R10m", "R10m", "R10m", "R10m", "R20m", "R20m", "R20m"] # e la relativa risoluzione
bands_and_resolutions = list(zip(bands, resolutions))

target_dim = (10980, 10980)

In [ ]:
src_data_dirs = []

In [ ]:
# per ogni giorno a disposizione individua il path in cui recuperare i dati
for x in os.listdir(src_root_data_dir):
    date = x.split("_")[2].split("T")[0]
    src_data_dir = glob(os.path.join(src_root_data_dir, x, "GRANULE/*/IMG_DATA"))[0]
    src_data_dirs.append((date, src_data_dir))

src_data_dirs = sorted(src_data_dirs, key=lambda x: x[0])

In [ ]:
for date, src_data_dir in src_data_dirs:
    tiff_file = os.path.join(tiff_root_data_dir, date + ".tiff")
    
    if os.path.exists(tiff_file):
        continue
    
    tiff_f = None
    
    for i, (band, resolution) in enumerate(bands_and_resolutions, start=1):
        band_file = glob(os.path.join(src_data_dir, resolution, "*_" + band + "_*.jp2"))[0]
        
        band_f = rasterio.open(band_file, driver="JP2OpenJPEG")
        band_data = band_f.read(1)

        if band_data.shape[0] < target_dim[0] and band_data.shape[1] < target_dim[1]:
            print("Extrapolating", band_data.shape, "to", target_dim)
            band_data = extrapolate(band_data, target_dim).astype(band_f.dtypes[0])
            
        if tiff_f is None:  
            profile = band_f.profile
            profile.update(driver="Gtiff", count=len(bands_and_resolutions))
            tiff_f = MemoryFile().open(**profile)
            
        print("Writing band {} for date {}".format(band, date))
        tiff_f.write(band_data, i)
        
        band_f.close()
        
        
    tiff_f_cropped = crop_memory_tiff_file(tiff_f, [get_test_polygon()], crop=True)
    
    tiff_f.close()
    tiff_f = None

    with rasterio.open(tiff_file, "w", **tiff_f_cropped.profile) as f:
        f.write(tiff_f_cropped.read())
        
    tiff_f_cropped.close()

In [ ]:
glob(tiff_root_data_dir + '/*')

In [ ]:
img = rasterio.open('..your tiff file..')

In [ ]:
# https://gis.stackexchange.com/questions/341809/merging-sentinel-2-rgb-bands-with-rasterio#355077
image = np.array([img.read(3), img.read(2), img.read(1)]).transpose(1,2,0)
p2, p98 = np.percentile(image, (2,98))
image = exposure.rescale_intensity(image, in_range=(p2, p98)) / 100000

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 8))
show(image.transpose(2,0,1), transform=img.transform, ax=ax, cmap='coolwarm_r')
plt.show()